Hi Kiren! This is a "notebook" that is useful for getting comfier with both Python + real-world research!

In [34]:
from pubmed_tools.core.client import PubMedClient
from pubmed_tools.parsers.article import ArticleParser
from pubmed_tools.exporters.pdf_exporter import PDFExporter
from pubmed_tools.exporters.csv_exporter import CSVExporter
import os
import subprocess

# Initialize components
client = PubMedClient()
parser = ArticleParser()

In [35]:
DEFAULT_QUERY = "chunking memory"
# Configuration settings
CONFIG = {
    'exporters': {
        'pdf': {
            'enabled': True,
            'auto_open': True,
            'exporter': PDFExporter(),
        },
        'csv': {
            'enabled': False,
            'auto_open': True,
            'exporter': CSVExporter(),
        },
    },
    'truncate_articles': {
        'enabled': False,
        'count': 2,
    }
}

def open_file(filename: str) -> None:
    """Opens a file using the system's default application.

    Args:
        filename: Path to the file to open

    Note:
        This is platform-dependent and is only tested on macOS.
    """
    if not os.path.exists(filename):
        raise FileNotFoundError(f"File not found: {filename}")

    valid_formats = ['.pdf', '.csv', '.xlsx']
    file_ext = os.path.splitext(filename.lower())[1]
    if file_ext not in valid_formats:
        raise ValueError(
            f"File must be one of {valid_formats}, got: {filename}")
    subprocess.Popen(f"open {filename}", shell=True)

In [36]:
query = DEFAULT_QUERY

# Search and fetch articles
results = client.search(query)
id_list = results["id_list"]
details = client.fetch_details(id_list)

# Parse articles
parsed_articles = parser.parse_all_details(details)
if not parsed_articles:
    print("No articles found")
elif CONFIG['truncate_articles']['enabled']:
    parsed_articles = parsed_articles[:CONFIG['truncate_articles']['count']]

In [37]:
# Export to different formats
exported_files = {}
for format_name, settings in CONFIG['exporters'].items():
    if settings['enabled']:
        filename = f"output.{format_name}"
        settings['exporter'].export(parsed_articles, filename)
        exported_files[format_name] = filename
        print(f"Exported to {filename}")

Exported to output.pdf


In [38]:
open_file(exported_files['pdf'])

In [39]:
parsed_articles

[{'title': 'Chunk Duration Limits the Learning of Multiword Chunks: Behavioral and Electroencephalogram Evidence from Statistical Learning.',
  'abstract': "Language comprehension involves the grouping of words into larger multiword chunks. This is required to recode information into sparser representations to mitigate memory limitations and counteract forgetting. It has been suggested that electrophysiological processing time windows constrain the formation of these units. Specifically, the period of rhythmic neural activity (i.e., slow-frequency neural oscillations) may set an upper limit of 2-3 sec. Here, we assess whether learning of new multiword chunks is also affected by this neural limit. We applied an auditory statistical learning paradigm of an artificial language while manipulating the duration of to-be-learnt chunks. Participants listened to isochronous sequences of disyllabic pseudowords from which they could learn hidden three-word chunks based on transitional probabiliti